# 📊 Export CSV per Presentazione Canva

Questo notebook estrae i dati chiave dall'analisi delle metriche e li esporta in formato CSV
ottimizzato per l'importazione diretta in Canva per mantenere uno stile coerente nella presentazione.

## 📋 File CSV generati:
1. **`metriche_per_anno.csv`** - Performance per anno (Top-1, Top-5, MRR)
2. **`confronto_modelli.csv`** - Confronto tra diversi modelli 
3. **`metriche_globali.csv`** - Performance aggregate generali
4. **`coverage_analysis.csv`** - Analisi della copertura del catalogo
5. **`yearly_summary.csv`** - Riassunto annuale con conteggi


In [5]:
# Importazioni necessarie
from pathlib import Path
import glob, ast, os
import numpy as np
import pandas as pd
import json

# Crea directory output per i CSV
output_dir = Path('../csv_for_canva')
output_dir.mkdir(exist_ok=True)

print(f"✅ Directory output creata: {output_dir}")
print(f"📁 I file CSV verranno salvati in: {output_dir.absolute()}")

✅ Directory output creata: ../csv_for_canva
📁 I file CSV verranno salvati in: /leonardo_work/IscrC_LLM-Mob/LLM-Mob-As-Mobility-Interpreter/notebook/../csv_for_canva


## 🔄 Caricamento Dati
Utilizziamo lo stesso codice del notebook principale per caricare e processare i dati

In [6]:
# Funzioni di parsing (copiate dal notebook principale)
def safe_parse_prediction(x, debug_mode=False):
    """
    Parsing sicuro delle predizioni da stringa a lista
    Gestisce vari formati inclusi dict e stringhe
    """
    if pd.isna(x) or x == "[]" or x == "" or not isinstance(x, str):
        return []
    
    try:
        parsed = ast.literal_eval(x)
        
        if isinstance(parsed, list):
            # Se è una lista, estrai le stringhe (gestisce sia str che dict)
            result = []
            for item in parsed:
                if isinstance(item, str):
                    result.append(item.strip())
                elif isinstance(item, dict):
                    # Se è un dict, prova a estrarre un campo chiave
                    if 'name' in item:
                        result.append(str(item['name']).strip())
                    elif 'poi' in item:
                        result.append(str(item['poi']).strip())
                    else:
                        # Prendi il primo valore del dict
                        values = list(item.values())
                        if values:
                            result.append(str(values[0]).strip())
                else:
                    # Converti altri tipi in stringa
                    result.append(str(item).strip())
            return result
        else:
            return []
    except Exception as e:
        if debug_mode:
            print(f"Errore parsing: {x} -> {e}")
        return []

def normalize_poi_name(poi_name):
    """
    Normalizza il nome di un POI per confronti consistenti
    """
    if pd.isna(poi_name) or not isinstance(poi_name, str):
        return str(poi_name).strip()
    
    # Rimuovi spazi extra e normalizza
    normalized = poi_name.strip()
    
    # Potrebbero esserci altre normalizzazioni specifiche
    # Es: rimuovere caratteri speciali, convertire a lowercase, etc.
    
    return normalized

def load_and_process_data(model_name, version="base_version"):
    """
    Carica e processa i dati per un modello specifico
    """
    # Path dei file CSV
    csv_pattern = f'../results/{model_name}/{version}/*_pred_*.csv'
    csv_files = [Path(p) for p in glob.glob(csv_pattern)]
    csv_files = sorted(csv_files)
    
    if not csv_files:
        print(f"❌ Nessun file trovato per {model_name}/{version}")
        return None
    
    print(f"📂 Trovati {len(csv_files)} file per {model_name}/{version}")
    
    # Lista per contenere tutti i DataFrame
    dfs = []
    total_processed = 0
    
    for fp in csv_files:
        print(f"  Processando {fp.name}...")
        try:
            df = pd.read_csv(fp)
        except pd.errors.ParserError:
            print(f"⚠️ Errore parsing {fp.name}, usando error handling...")
            df = pd.read_csv(fp, on_bad_lines='skip', engine='python')
        
        print(f"    Righe caricate: {len(df)}")
        
        # Estrai anno dal nome file
        year_token = next((part for part in fp.stem.split('_') 
                          if part.isdigit() and len(part) == 4), None)
        if year_token:
            df['year'] = int(year_token)
        else:
            print(f"⚠️ Anno non trovato in {fp.name}")
            continue
        
        # Parsing predizioni con gestione errori migliorata
        df['prediction_list'] = df['prediction'].apply(safe_parse_prediction)
        
        # Debug: mostra alcuni esempi di parsing
        valid_predictions = df[df['prediction_list'].apply(lambda x: len(x) > 0)]
        invalid_predictions = df[df['prediction_list'].apply(lambda x: len(x) == 0)]
        
        print(f"    Predizioni valide: {len(valid_predictions)}")
        print(f"    Predizioni invalide: {len(invalid_predictions)}")
        
        # Filtra righe valide (con predizioni non vuote)
        df = df[df['prediction_list'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
        
        if len(df) > 0:
            dfs.append(df)
            total_processed += len(df)
            print(f"    ✅ Righe valide aggiunte: {len(df)}")
        else:
            print(f"    ❌ Nessuna riga valida in {fp.name}")
    
    if not dfs:
        print(f"❌ Nessun dato valido trovato per {model_name}")
        return None
    
    # Concatena tutti i DataFrame
    df_all = pd.concat(dfs, ignore_index=True)
    print(f"✅ Dataset combinato: {len(df_all):,} righe")
    
    # Normalizzazione POI (migliorata)
    df_all['prediction_norm'] = df_all['prediction_list'].apply(
        lambda x: [normalize_poi_name(poi) for poi in x] if isinstance(x, list) else []
    )
    df_all['ground_truth_norm'] = df_all['ground_truth'].apply(normalize_poi_name)
    
    return df_all

print("✅ Funzioni di caricamento dati definite (versione corretta)")

✅ Funzioni di caricamento dati definite (versione corretta)


In [7]:
def calculate_metrics(df):
    """
    Calcola le metriche per un dataset
    """
    def safe_top1_accuracy(row):
        pred_norm = row['prediction_norm']
        if not pred_norm or len(pred_norm) == 0:
            return False
        return pred_norm[0] == row['ground_truth_norm']
    
    def safe_top_k_hit(row, k=5):
        pred_norm = row['prediction_norm']
        if not pred_norm or len(pred_norm) == 0:
            return False
        return row['ground_truth_norm'] in pred_norm[:k]
    
    def safe_reciprocal_rank(row, k=5):
        pred_norm = row['prediction_norm']
        if not pred_norm or len(pred_norm) == 0:
            return 0.0
        try:
            rank = pred_norm[:k].index(row['ground_truth_norm']) + 1
            return 1.0 / rank
        except ValueError:
            return 0.0
    
    # Calcola metriche
    df['hit@1'] = df.apply(safe_top1_accuracy, axis=1)
    df['hit@5'] = df.apply(safe_top_k_hit, axis=1)
    df['rr'] = df.apply(safe_reciprocal_rank, axis=1)
    
    return df

print("✅ Funzioni di calcolo metriche definite")

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
✅ Funzioni di calcolo metriche definite


## 🤖 Processamento Modelli
Carichiamo e processiamo i dati per tutti i modelli disponibili

In [8]:
# Lista dei modelli da processare (in ordine di priorità)
models_to_process = [
    'qwen2.5_14b',
    'deepseek-coder_33b', 
    'qwen2.5_7b',
    'llama3.1_8b',
    'mixtral_8x7b',
    'deepseek-r1_32b'  # Aggiungo anche questo se presente
]

# Dizionario per contenere i dati di tutti i modelli
models_data = {}
failed_models = []

# Carica dati per ogni modello
for model in models_to_process:
    print(f"\n🔄 Processando {model}...")
    try:
        df = load_and_process_data(model, "base_version")
        if df is not None and len(df) > 0:
            df = calculate_metrics(df)
            df['model'] = model  # Aggiungi colonna modello
            models_data[model] = df
            print(f"✅ {model}: {len(df):,} predizioni caricate")
            
            # Mostra alcune statistiche di base
            years = sorted(df['year'].unique())
            print(f"   📅 Anni coperti: {years[0]} - {years[-1]} ({len(years)} anni)")
            print(f"   🎯 Top-1 Accuracy: {df['hit@1'].mean():.1%}")
        else:
            print(f"❌ {model}: Dataset vuoto o errore nel caricamento")
            failed_models.append(model)
    except Exception as e:
        print(f"❌ {model}: Errore - {str(e)}")
        failed_models.append(model)

print(f"\n📊 RIEPILOGO CARICAMENTO:")
print(f"✅ Modelli caricati con successo: {len(models_data)}")
print(f"❌ Modelli falliti: {len(failed_models)}")

if failed_models:
    print(f"   Modelli falliti: {', '.join(failed_models)}")

# Crea un dataset combinato per alcune analisi (solo se ci sono dati)
if models_data:
    try:
        combined_df = pd.concat(models_data.values(), ignore_index=True)
        print(f"📈 Dataset combinato: {len(combined_df):,} predizioni totali")
        print(f"🤖 Modelli nel dataset combinato: {', '.join(models_data.keys())}")
        
        # Verifica la distribuzione per modello
        model_counts = combined_df['model'].value_counts()
        print("📊 Distribuzione predizioni per modello:")
        for model, count in model_counts.items():
            print(f"   {model}: {count:,}")
            
    except Exception as e:
        print(f"⚠️ Errore nella creazione del dataset combinato: {e}")
        combined_df = None
else:
    combined_df = None
    print("❌ Nessun modello caricato con successo")
    print("🔧 Suggerimenti:")
    print("   - Verifica che i file CSV esistano nella directory ../results/")
    print("   - Controlla che i path dei modelli siano corretti")
    print("   - Verifica la struttura dei file CSV")


🔄 Processando qwen2.5_14b...
📂 Trovati 12 file per qwen2.5_14b/base_version
  Processando dati_2014_pred_20250829_013511.csv...
    Righe caricate: 65891
    Predizioni valide: 65891
    Predizioni invalide: 0
    ✅ Righe valide aggiunte: 65891
  Processando dati_2015_pred_20250829_071654.csv...
    Righe caricate: 66672
    Predizioni valide: 66672
    Predizioni invalide: 0
    ✅ Righe valide aggiunte: 66672
  Processando dati_2016_pred_20250829_130518.csv...
    Righe caricate: 70875
    Predizioni valide: 70875
    Predizioni invalide: 0
    ✅ Righe valide aggiunte: 70875
  Processando dati_2017_pred_20250829_191315.csv...
    Righe caricate: 81342
    Predizioni valide: 81342
    Predizioni invalide: 0
    ✅ Righe valide aggiunte: 81342
  Processando dati_2018_pred_20250830_021506.csv...
    Righe caricate: 78382
    Predizioni valide: 78382
    Predizioni invalide: 0
    ✅ Righe valide aggiunte: 78382
  Processando dati_2019_pred_20250830_090457.csv...
    Righe caricate: 71224


## 📤 Esportazione CSV per Canva
Generiamo tutti i file CSV ottimizzati per Canva

In [9]:
# 1. METRICHE PER ANNO (usando il modello migliore disponibile)
if models_data:
    # Scegli il modello principale in ordine di priorità
    primary_model = None
    priority_order = ['qwen2.5_14b', 'deepseek-coder_33b', 'qwen2.5_7b', 'llama3.1_8b', 'mixtral_8x7b']
    
    for preferred_model in priority_order:
        if preferred_model in models_data:
            primary_model = preferred_model
            break
    
    # Se nessun modello prioritario, prendi il primo disponibile
    if primary_model is None:
        primary_model = list(models_data.keys())[0]
    
    df_primary = models_data[primary_model]
    print(f"🎯 Usando {primary_model} come modello principale per metriche annuali")
    
    try:
        # Calcola metriche per anno
        yearly_metrics = df_primary.groupby('year').agg({
            'hit@1': 'mean',
            'hit@5': 'mean', 
            'rr': 'mean',
            'card_id': 'count'
        }).round(4)
        
        # Rinomina colonne per Canva
        yearly_metrics.columns = ['Top-1_Accuracy', 'Top-5_Hit_Rate', 'MRR', 'Num_Predictions']
        
        # Converti percentuali
        yearly_metrics['Top-1_Accuracy_Percent'] = (yearly_metrics['Top-1_Accuracy'] * 100).round(2)
        yearly_metrics['Top-5_Hit_Rate_Percent'] = (yearly_metrics['Top-5_Hit_Rate'] * 100).round(2) 
        yearly_metrics['MRR_Percent'] = (yearly_metrics['MRR'] * 100).round(2)
        
        # Aggiungi informazioni sul modello
        yearly_metrics['Model_Used'] = primary_model
        yearly_metrics['Model_Display'] = primary_model.replace('_', ' ').replace('-', ' ').title()
        
        # Reset index per avere year come colonna
        yearly_metrics = yearly_metrics.reset_index()
        
        # Aggiungi medie per riferimento
        overall_means = {
            'year': 'MEDIA',
            'Top-1_Accuracy': yearly_metrics['Top-1_Accuracy'].mean(),
            'Top-5_Hit_Rate': yearly_metrics['Top-5_Hit_Rate'].mean(),
            'MRR': yearly_metrics['MRR'].mean(),
            'Num_Predictions': yearly_metrics['Num_Predictions'].sum(),
            'Top-1_Accuracy_Percent': yearly_metrics['Top-1_Accuracy_Percent'].mean(),
            'Top-5_Hit_Rate_Percent': yearly_metrics['Top-5_Hit_Rate_Percent'].mean(),
            'MRR_Percent': yearly_metrics['MRR_Percent'].mean(),
            'Model_Used': primary_model,
            'Model_Display': yearly_metrics['Model_Display'].iloc[0]
        }
        
        # Aggiungi la riga media
        yearly_metrics = pd.concat([yearly_metrics, pd.DataFrame([overall_means])], ignore_index=True)
        
        # Esporta
        yearly_csv = output_dir / 'metriche_per_anno.csv'
        yearly_metrics.to_csv(yearly_csv, index=False)
        
        print(f"✅ Esportato: {yearly_csv}")
        print(f"📊 Dati per {len(yearly_metrics)-1} anni + riga media (modello: {primary_model})")
        print("📋 Prime righe:")
        display(yearly_metrics.head())
        
    except Exception as e:
        print(f"❌ Errore nell'esportazione metriche annuali: {e}")
        print("🔧 Controllare che il dataset contenga colonne year, hit@1, hit@5, rr, card_id")
else:
    print("⚠️ Nessun modello disponibile per le metriche annuali")

🎯 Usando qwen2.5_14b come modello principale per metriche annuali
✅ Esportato: ../csv_for_canva/metriche_per_anno.csv
📊 Dati per 10 anni + riga media (modello: qwen2.5_14b)
📋 Prime righe:


,year,Top-1_Accuracy,Top-5_Hit_Rate,MRR,Num_Predictions,Top-1_Accuracy_Percent,Top-5_Hit_Rate_Percent,MRR_Percent,Model_Used,Model_Display
0,2014,0.0318,0.2427,0.0983,65891,3.18,24.27,9.83,qwen2.5_14b,Qwen2.5 14B
1,2015,0.0319,0.2506,0.1022,66672,3.19,25.06,10.22,qwen2.5_14b,Qwen2.5 14B
2,2016,0.0309,0.2570,0.1024,70875,3.09,25.70,10.24,qwen2.5_14b,Qwen2.5 14B
3,2017,0.0285,0.2735,0.1023,81342,2.85,27.35,10.23,qwen2.5_14b,Qwen2.5 14B
4,2018,0.0282,0.2703,0.1015,78382,2.82,27.03,10.15,qwen2.5_14b,Qwen2.5 14B


In [10]:
# 2. CONFRONTO TRA MODELLI
if len(models_data) > 1:
    print("🔄 Generando confronto tra modelli...")
    try:
        model_comparison = []
        
        for model_name, df in models_data.items():
            # Verifica che il DataFrame contenga le colonne necessarie
            required_cols = ['hit@1', 'hit@5', 'rr', 'year']
            missing_cols = [col for col in required_cols if col not in df.columns]
            
            if missing_cols:
                print(f"⚠️ {model_name}: colonne mancanti {missing_cols}, skip")
                continue
            
            # Calcola metriche globali per ogni modello
            metrics = {
                'Model': model_name.replace('_', ' ').replace('-', ' ').title(),
                'Model_Code': model_name,
                'Top-1_Accuracy': df['hit@1'].mean() if len(df) > 0 else 0,
                'Top-5_Hit_Rate': df['hit@5'].mean() if len(df) > 0 else 0,
                'MRR': df['rr'].mean() if len(df) > 0 else 0,
                'Total_Predictions': len(df),
                'Years_Covered': len(df['year'].unique()) if 'year' in df.columns else 0,
                'Year_Range': f"{df['year'].min()}-{df['year'].max()}" if 'year' in df.columns and len(df) > 0 else "N/A"
            }
            
            # Converti in percentuali
            metrics['Top-1_Accuracy_Percent'] = round(metrics['Top-1_Accuracy'] * 100, 2)
            metrics['Top-5_Hit_Rate_Percent'] = round(metrics['Top-5_Hit_Rate'] * 100, 2)
            metrics['MRR_Percent'] = round(metrics['MRR'] * 100, 2)
            
            # Aggiungi metriche di qualità
            metrics['Error_Rate_Percent'] = round(100 - metrics['Top-1_Accuracy_Percent'], 2)
            metrics['Predictions_Per_Year'] = round(metrics['Total_Predictions'] / max(1, metrics['Years_Covered']), 0)
            
            model_comparison.append(metrics)
        
        if model_comparison:
            # Crea DataFrame e ordina per Top-1 Accuracy
            model_comparison_df = pd.DataFrame(model_comparison)
            model_comparison_df = model_comparison_df.sort_values('Top-1_Accuracy_Percent', ascending=False)
            
            # Aggiungi ranking
            model_comparison_df['Rank_Top1'] = range(1, len(model_comparison_df) + 1)
            model_comparison_df['Rank_Top5'] = model_comparison_df['Top-5_Hit_Rate_Percent'].rank(ascending=False, method='min').astype(int)
            model_comparison_df['Rank_MRR'] = model_comparison_df['MRR_Percent'].rank(ascending=False, method='min').astype(int)
            
            # Aggiungi classificazioni qualitative
            model_comparison_df['Performance_Level'] = pd.cut(
                model_comparison_df['Top-1_Accuracy_Percent'],
                bins=[0, 3, 5, 7, 100],
                labels=['Base', 'Good', 'Very Good', 'Excellent']
            )
            
            # Esporta
            comparison_csv = output_dir / 'confronto_modelli.csv'
            model_comparison_df.to_csv(comparison_csv, index=False)
            
            print(f"✅ Esportato: {comparison_csv}")
            print(f"🤖 Confronto di {len(model_comparison_df)} modelli")
            print("🏆 Top 3 modelli per Top-1 Accuracy:")
            top_models = model_comparison_df.head(3)[['Model', 'Top-1_Accuracy_Percent', 'Rank_Top1']]
            for _, row in top_models.iterrows():
                print(f"   {row['Rank_Top1']}. {row['Model']}: {row['Top-1_Accuracy_Percent']}%")
            
            display(model_comparison_df[['Model', 'Top-1_Accuracy_Percent', 'Top-5_Hit_Rate_Percent', 'MRR_Percent', 'Performance_Level']].head())
        else:
            print("❌ Nessun modello valido per il confronto")
            
    except Exception as e:
        print(f"❌ Errore nella generazione confronto modelli: {e}")
        print("🔧 Verificare che i modelli abbiano le colonne hit@1, hit@5, rr")

elif len(models_data) == 1:
    print("⚠️ Un solo modello disponibile, creando file di confronto con singolo modello")
    try:
        single_model = list(models_data.keys())[0]
        df = models_data[single_model]
        
        single_model_data = [{
            'Model': single_model.replace('_', ' ').replace('-', ' ').title(),
            'Model_Code': single_model,
            'Top-1_Accuracy_Percent': round(df['hit@1'].mean() * 100, 2),
            'Top-5_Hit_Rate_Percent': round(df['hit@5'].mean() * 100, 2),
            'MRR_Percent': round(df['rr'].mean() * 100, 2),
            'Total_Predictions': len(df),
            'Years_Covered': len(df['year'].unique()),
            'Rank_Top1': 1,
            'Performance_Level': 'Single Model',
            'Note': 'Only model available'
        }]
        
        single_model_df = pd.DataFrame(single_model_data)
        comparison_csv = output_dir / 'confronto_modelli.csv'
        single_model_df.to_csv(comparison_csv, index=False)
        print(f"✅ Esportato confronto singolo modello: {comparison_csv}")
        
    except Exception as e:
        print(f"❌ Errore nell'esportazione singolo modello: {e}")
else:
    print("❌ Nessun modello disponibile per il confronto")

🔄 Generando confronto tra modelli...
✅ Esportato: ../csv_for_canva/confronto_modelli.csv
🤖 Confronto di 5 modelli
🏆 Top 3 modelli per Top-1 Accuracy:
   1. Llama3.1 8B: 10.05%
   2. Mixtral 8X7B: 3.61%
   3. Qwen2.5 14B: 2.89%


,Model,Top-1_Accuracy_Percent,Top-5_Hit_Rate_Percent,MRR_Percent,Performance_Level
3,Llama3.1 8B,10.05,16.23,12.58,Excellent
4,Mixtral 8X7B,3.61,21.35,11.18,Good
0,Qwen2.5 14B,2.89,26.18,10.09,Base
1,Deepseek Coder 33B,2.32,21.93,8.36,Base
2,Qwen2.5 7B,0.00,0.25,0.11,NaN


In [11]:
# 3. METRICHE GLOBALI (aggregate su tutti i modelli o modello principale)
if combined_df is not None:
    # Usa il dataset combinato se disponibile
    df_for_global = combined_df
    scope = "Tutti i Modelli"
else:
    # Usa il modello principale
    df_for_global = df_primary
    scope = f"Modello {primary_model}"

# Calcola metriche globali
global_metrics = {
    'Metric': ['Top-1 Accuracy', 'Top-5 Hit Rate', 'Mean Reciprocal Rank', 'Total Predictions'],
    'Value': [
        df_for_global['hit@1'].mean(),
        df_for_global['hit@5'].mean(), 
        df_for_global['rr'].mean(),
        len(df_for_global)
    ],
    'Percentage': [
        round(df_for_global['hit@1'].mean() * 100, 2),
        round(df_for_global['hit@5'].mean() * 100, 2),
        round(df_for_global['rr'].mean() * 100, 2),
        len(df_for_global)  # Mantieni conteggio come numero
    ],
    'Scope': [scope] * 4
}

global_metrics_df = pd.DataFrame(global_metrics)

# Esporta
global_csv = output_dir / 'metriche_globali.csv'
global_metrics_df.to_csv(global_csv, index=False)

print(f"✅ Esportato: {global_csv}")
print(f"🌍 Scope: {scope}")
display(global_metrics_df)

✅ Esportato: ../csv_for_canva/metriche_globali.csv
🌍 Scope: Tutti i Modelli


,Metric,Value,Percentage,Scope
0,Top-1 Accuracy,4.236147e-02,4.24,Tutti i Modelli
1,Top-5 Hit Rate,1.806913e-01,18.07,Tutti i Modelli
2,Mean Reciprocal Rank,9.084302e-02,9.08,Tutti i Modelli
3,Total Predictions,3.070314e+06,3070314.00,Tutti i Modelli


In [12]:
# 4. ANALISI COVERAGE
if models_data:
    coverage_analysis = []
    
    for model_name, df in models_data.items():
        # Calcola POI unici
        all_predicted_pois = set()
        for pred_list in df['prediction_norm']:
            if pred_list:
                all_predicted_pois.update(pred_list)
        
        ground_truth_pois = set(df['ground_truth_norm'].unique())
        
        # Calcola metriche di coverage
        overlap = all_predicted_pois.intersection(ground_truth_pois)
        coverage = len(all_predicted_pois) / len(ground_truth_pois) if len(ground_truth_pois) > 0 else 0
        recall_coverage = len(overlap) / len(ground_truth_pois) if len(ground_truth_pois) > 0 else 0
        precision_coverage = len(overlap) / len(all_predicted_pois) if len(all_predicted_pois) > 0 else 0
        
        coverage_data = {
            'Model': model_name.replace('_', ' ').replace('-', ' ').title(),
            'Model_Code': model_name,
            'POI_Predicted_Unique': len(all_predicted_pois),
            'POI_GroundTruth_Unique': len(ground_truth_pois),
            'POI_Overlap': len(overlap),
            'Catalogue_Coverage': round(coverage * 100, 2),
            'Coverage_Recall': round(recall_coverage * 100, 2),
            'Coverage_Precision': round(precision_coverage * 100, 2)
        }
        
        coverage_analysis.append(coverage_data)
    
    coverage_df = pd.DataFrame(coverage_analysis)
    
    # Esporta
    coverage_csv = output_dir / 'coverage_analysis.csv'
    coverage_df.to_csv(coverage_csv, index=False)
    
    print(f"✅ Esportato: {coverage_csv}")
    print(f"📋 Analisi coverage per {len(coverage_df)} modelli")
    display(coverage_df)

✅ Esportato: ../csv_for_canva/coverage_analysis.csv
📋 Analisi coverage per 5 modelli


,Model,Model_Code,POI_Predicted_Unique,POI_GroundTruth_Unique,POI_Overlap,Catalogue_Coverage,Coverage_Recall,Coverage_Precision
0,Qwen2.5 14B,qwen2.5_14b,1433,22,16,6513.64,72.73,1.12
1,Deepseek Coder 33B,deepseek-coder_33b,2187,22,22,9940.91,100.00,1.01
2,Qwen2.5 7B,qwen2.5_7b,107559,22,21,488904.55,95.45,0.02
3,Llama3.1 8B,llama3.1_8b,28780,22,19,130818.18,86.36,0.07
4,Mixtral 8X7B,mixtral_8x7b,1329,22,18,6040.91,81.82,1.35


In [13]:
# 5. RIASSUNTO ANNUALE DETTAGLIATO
if models_data:
    # Usa il dataset combinato se disponibile, altrimenti il modello principale
    df_for_yearly = combined_df if combined_df is not None else df_primary
    
    # Calcola statistiche dettagliate per anno
    yearly_summary = df_for_yearly.groupby('year').agg({
        'hit@1': ['count', 'mean', 'std'],
        'hit@5': ['mean', 'std'],
        'rr': ['mean', 'std'],
        'card_id': 'nunique'  # Numero di utenti unici
    }).round(4)
    
    # Flattening delle colonne
    yearly_summary.columns = [
        'Total_Predictions', 'Top1_Mean', 'Top1_Std',
        'Top5_Mean', 'Top5_Std', 'MRR_Mean', 'MRR_Std', 'Unique_Users'
    ]
    
    # Converti in percentuali
    for col in ['Top1_Mean', 'Top1_Std', 'Top5_Mean', 'Top5_Std', 'MRR_Mean', 'MRR_Std']:
        yearly_summary[f'{col}_Percent'] = (yearly_summary[col] * 100).round(2)
    
    # Calcola tasso di errore
    yearly_summary['Error_Rate_Percent'] = (100 - yearly_summary['Top1_Mean_Percent']).round(2)
    
    # Reset index
    yearly_summary = yearly_summary.reset_index()
    
    # Aggiungi informazioni aggiuntive
    yearly_summary['Data_Source'] = 'Combined Models' if combined_df is not None else primary_model
    
    # Esporta
    summary_csv = output_dir / 'yearly_summary.csv'
    yearly_summary.to_csv(summary_csv, index=False)
    
    print(f"✅ Esportato: {summary_csv}")
    print(f"📅 Riassunto per {len(yearly_summary)} anni")
    
    # Mostra colonne principali
    display(yearly_summary[['year', 'Total_Predictions', 'Top1_Mean_Percent', 
                           'Top5_Mean_Percent', 'MRR_Mean_Percent', 'Error_Rate_Percent']])

✅ Esportato: ../csv_for_canva/yearly_summary.csv
📅 Riassunto per 10 anni


,year,Total_Predictions,Top1_Mean_Percent,Top5_Mean_Percent,MRR_Mean_Percent,Error_Rate_Percent
0,2014,374796,4.65,14.09,8.22,95.35
1,2015,379110,5.01,15.47,8.93,94.99
2,2016,333289,4.13,19.09,9.46,95.87
3,2017,380977,3.93,19.47,9.17,96.07
4,2018,371171,3.90,19.23,9.15,96.10
5,2019,668324,4.37,20.03,9.77,95.63
6,2020,65472,5.12,21.81,10.96,94.88
7,2021,103542,3.37,16.21,7.76,96.63
8,2022,343373,3.64,17.47,8.43,96.36
9,2023,50260,3.86,18.34,8.74,96.14


## 📋 Riepilogo File Esportati
Verifica finale dei file CSV generati

In [14]:
# Verifica file esportati
csv_files = list(output_dir.glob('*.csv'))

print("📊 FILE CSV ESPORTATI PER CANVA:")
print("==" * 25)

if csv_files:
    total_size = 0
    for csv_file in sorted(csv_files):
        try:
            df_check = pd.read_csv(csv_file)
            file_size = csv_file.stat().st_size
            total_size += file_size
            
            print(f"✅ {csv_file.name}")
            print(f"   📏 Dimensioni: {df_check.shape[0]} righe × {df_check.shape[1]} colonne")
            print(f"   💾 Dimensione file: {file_size:,} bytes ({file_size/1024:.1f} KB)")
            print(f"   📂 Path: {csv_file}")
            
            # Mostra le prime colonne per verifica
            columns_preview = ', '.join(df_check.columns[:5])
            if len(df_check.columns) > 5:
                columns_preview += f", ... (+{len(df_check.columns)-5} altre)"
            print(f"   📋 Colonne: {columns_preview}")
            print()
            
        except Exception as e:
            print(f"❌ {csv_file.name}: Errore nella lettura - {e}")
            print()

    print(f"🎯 TOTALE: {len(csv_files)} file CSV pronti per Canva")
    print(f"💾 Dimensione totale: {total_size:,} bytes ({total_size/1024:.1f} KB)")
    print(f"📁 Directory: {output_dir.absolute()}")
    
    # Verifica integrità dei dati
    print("\n🔍 VERIFICA INTEGRITÀ DATI:")
    print("=" * 30)
    
    integrity_issues = []
    
    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file)
            issues = []
            
            # Controlla valori mancanti
            missing_values = df.isnull().sum().sum()
            if missing_values > 0:
                issues.append(f"{missing_values} valori mancanti")
            
            # Controlla colonne percentuali
            pct_cols = [col for col in df.columns if 'Percent' in col]
            for col in pct_cols:
                if col in df.columns:
                    invalid_pcts = ((df[col] < 0) | (df[col] > 100)).sum()
                    if invalid_pcts > 0:
                        issues.append(f"{invalid_pcts} percentuali invalide in {col}")
            
            if issues:
                integrity_issues.append(f"{csv_file.name}: {', '.join(issues)}")
            else:
                print(f"✅ {csv_file.name}: Dati integri")
                
        except Exception as e:
            integrity_issues.append(f"{csv_file.name}: Errore verifica - {e}")
    
    if integrity_issues:
        print("\n⚠️ PROBLEMI RILEVATI:")
        for issue in integrity_issues:
            print(f"   {issue}")
    else:
        print("✅ Tutti i file hanno superato la verifica di integrità")

    # Suggerimenti per Canva
    print("\n💡 SUGGERIMENTI PER CANVA:")
    print("=" * 30)
    
    suggestions = {
        'metriche_per_anno.csv': [
            "📈 Grafici temporali (line chart con anni sull'asse X)",
            "📊 Bar chart per confronti annuali",
            "🎯 Usa colonne *_Percent per percentuali già formattate"
        ],
        'confronto_modelli.csv': [
            "📊 Horizontal bar chart per ranking modelli",
            "🏆 Usa Rank_Top1 per ordinamento",
            "📋 Tabelle comparative con Performance_Level"
        ],
        'metriche_globali.csv': [
            "🎯 KPI cards per dashboard",
            "📊 Gauge charts con colonna Percentage",
            "📈 Metriche principali per overview"
        ],
        'coverage_analysis.csv': [
            "📊 Grafici a barre per copertura",
            "🎯 Scatter plot Precision vs Recall",
            "📋 Tabelle dettagliate per appendice"
        ],
        'yearly_summary.csv': [
            "📊 Tabelle complete con statistiche",
            "📈 Include deviazioni standard",
            "🔍 Dati dettagliati per analisi profonde"
        ]
    }
    
    for csv_file in csv_files:
        file_name = csv_file.name
        if file_name in suggestions:
            print(f"\n📄 {file_name}:")
            for suggestion in suggestions[file_name]:
                print(f"   {suggestion}")
    
    print(f"\n🎨 FORMATO DATI OTTIMIZZATO:")
    print("   • Percentuali già convertite (es: 4.32 invece di 0.0432)")
    print("   • Nomi modelli formattati per presentazione")
    print("   • Encoding UTF-8 compatibile con Canva")
    print("   • Struttura pulita senza indici complessi")
    print("   • Colonne separate per valori assoluti e percentuali")
    
    print(f"\n🚀 PRONTO PER CANVA! Importa i file e crea visualizzazioni fantastiche!")

else:
    print("❌ Nessun file CSV trovato!")
    print("\n🔧 RISOLUZIONE PROBLEMI:")
    print("   1. Verifica che almeno un modello sia stato caricato correttamente")
    print("   2. Controlla gli errori nelle sezioni di esportazione precedenti")
    print("   3. Verifica che la directory di output sia accessibile")
    print("   4. Re-esegui le celle di esportazione")

📊 FILE CSV ESPORTATI PER CANVA:
✅ confronto_modelli.csv
   📏 Dimensioni: 5 righe × 17 colonne
   💾 Dimensione file: 989 bytes (1.0 KB)
   📂 Path: ../csv_for_canva/confronto_modelli.csv
   📋 Colonne: Model, Model_Code, Top-1_Accuracy, Top-5_Hit_Rate, MRR, ... (+12 altre)

✅ coverage_analysis.csv
   📏 Dimensioni: 5 righe × 8 colonne
   💾 Dimensione file: 418 bytes (0.4 KB)
   📂 Path: ../csv_for_canva/coverage_analysis.csv
   📋 Colonne: Model, Model_Code, POI_Predicted_Unique, POI_GroundTruth_Unique, POI_Overlap, ... (+3 altre)

✅ metriche_globali.csv
   📏 Dimensioni: 4 righe × 4 colonne
   💾 Dimensione file: 259 bytes (0.3 KB)
   📂 Path: ../csv_for_canva/metriche_globali.csv
   📋 Colonne: Metric, Value, Percentage, Scope

✅ metriche_per_anno.csv
   📏 Dimensioni: 11 righe × 10 colonne
   💾 Dimensione file: 998 bytes (1.0 KB)
   📂 Path: ../csv_for_canva/metriche_per_anno.csv
   📋 Colonne: year, Top-1_Accuracy, Top-5_Hit_Rate, MRR, Num_Predictions, ... (+5 altre)

✅ yearly_summary.csv
   📏 